In [1]:
import pyomo.environ as pyEnv
import math

In [2]:
#Model
model = pyEnv.ConcreteModel()

In [3]:
#Variables
N = 24 #number of nodes
M = 8  #number of UNLs
K = 4 #number of layers

# T = 11 #Trust thresold

U = N*(1/3) #Minimum UNL size
P = round(N*0.8) #Minimum overlap between every 2 UNLs

In [4]:
#Define layers

L = [6, 4, 2, 1] #Maximum number of UNLs a node can be part of
Q = [8, 4, 2, 1]  #Maximum number of nodes from a layer that a UNL can have

#Layers of each node
layers = [1,1,1,1,4,4,4,4,2,2,2,2,2,2,2,3,3,3,3,3,4,2,1,3]

In [5]:
#Indexes 
model.M = pyEnv.RangeSet(1,M)
model.N = pyEnv.RangeSet(1,N)
model.K = pyEnv.RangeSet(1,K)

In [6]:
#Params
# model.T = pyEnv.Param(within=pyEnv.NonNegativeIntegers, initialize=T, mutable=True)
model.U = pyEnv.Param(within=pyEnv.NonNegativeIntegers, initialize=U, mutable=True)
model.P = pyEnv.Param(within=pyEnv.NonNegativeIntegers, initialize=P, mutable=True)

model.L = pyEnv.Param(model.K,initialize=lambda model, i:L[i-1], mutable=True)
model.Q = pyEnv.Param(model.K,initialize=lambda model, i:Q[i-1], mutable=True)

def validate_y(model, value, aux):
    return value in model.K

model.y = pyEnv.Param(model.N, validate=validate_y, initialize=lambda model, i:layers[i-1])

In [7]:
#Decision variables
model.x = pyEnv.Var(model.N,model.M, within=pyEnv.Binary)
# model.U = pyEnv.Var(within=pyEnv.NonNegativeIntegers)

In [8]:
#Objective Function
# def obj_func(model):
#     return sum(model.x[i,j] * model.y[i] for i in model.N for j in model.M)

# model.objective1 = pyEnv.Objective(rule=obj_func,sense=pyEnv.maximize)

def obj_func(model):
    return sum(model.x[i,j]*model.y[i] for i in model.N for j in model.M)/M

model.objective = pyEnv.Objective(rule=obj_func,sense=pyEnv.minimize)

In [9]:
#Constraints

#Min UNL size
def unl_size(model,M):
    return sum(model.x[i,M] for i in model.N) >= model.U

#Min UNL total trust
# def unl_trust(model, M):
#     return sum(model.x[i,M]*model.y[i] for i in model.N) >= model.T

#Min overlap between every two UNLs
def min_overlap(model, M):
    return sum(model.x[i,M]*model.x[i,q] for i in model.N for q in model.M) >= model.P

#Max number of UNLs a node can be part
def max_node(model, N):
    return sum(model.x[N,j] for j in model.M) <= model.L[model.y[N]]

#Max number of nodes of a given layer in a UNL
model.max_layer = pyEnv.ConstraintList()
for j in model.M:
    for z in model.K:
        model.max_layer.add(sum(model.x[i,j] for i in model.N if (model.y[i] == z)) <= Q[z-1])

#Add
model.unl_size = pyEnv.Constraint(model.M,rule=unl_size)
# model.unl_trust = pyEnv.Constraint(model.M,rule=unl_trust)
model.min_overlap = pyEnv.Constraint(model.M,rule=min_overlap)
model.max_node = pyEnv.Constraint(model.N,rule=max_node)

In [10]:
#Prints the entire model
model.pprint()

2 Set Declarations
    max_layer_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   32 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    N*M :  192 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (10, 

In [11]:

path = "/home/flav/cplex/cplex/bin/x86-64_linux/cplex"

solver = pyEnv.SolverFactory('cplex',executable=path)
result = solver.solve(model,tee = True)
print('\n-------------------------\n')
print(result)

lis = list(model.x.keys())
for i in lis:
    # if model.x[i]() != 0:
    print(i,'--', model.x[i]())


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/tmp/tmpvz_kboyr.cplex.log' open.
CPLEX> Problem '/tmp/tmp9jsvjz0r.pyomo.lp' read.
Read time = 0.00 sec. (0.11 ticks)
CPLEX> Problem name         : /tmp/tmp9jsvjz0r.pyomo.lp
Objective sense      : Minimize
Variables            :     193  [Nneg: 1,  Binary: 192]
Objective nonzeros   :     192
Linear constraints   :      65  [Less: 56,  Greater: 8,  Equal: 1]
  Nonzeros           :     577
  RHS nonzeros       :      65
Quadratic constraints:       8  [Greater: 8]
  Linear terms       :       0
  Quadratic terms    :    1536
  RHS nonzeros       :       8

Variables            : Min LB: 0.000000